In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk, itertools, requests, zipfile, os

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
from itertools import product
from time import time 

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ritar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ritar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Download e descompactação do corpus disponibilizado pelo Professor: [text8](http://mattmahoney.net/dc/text8.zip)

O download do link foi realizado pela biblioteca **requests**, para a descompactação do arquivo zip foi utilizada a biblioteca **zipfile**. Por fim o conteudo do arquivo é lido e armazenado em uma variável *corpus* como uma cadeia de caractéres.

In [6]:
def get_corpus(path, file_name, url):
    
    if not os.path.isfile(path + 'text8'):
        try:
            r = requests.get(url)
            open(path + file_name , 'wb').write(r.content)
            with zipfile.ZipFile(path + file_name, 'r') as zip_ref:
                zip_ref.extractall(path)
        except (requests.exceptions.ConnectionError, requests.exceptions.HTTPError, 
                requests.exceptions.Timeout, requests.exceptions.RequestException) as err:
            print ("Error Connecting:",err)  
            return None

    with open(path + 'text8', 'r') as file:
        data = file.read().replace('\n', '')
    
    return data

corpus = get_corpus('./data/', 'text8.zip', 'http://mattmahoney.net/dc/text8.zip')
print(corpus[:200])

 anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst the term 


## Download de analogias disponibilizadas pelo Professor: [analogias](https://github.com/nicholas-leonard/word2vec/blob/master/questions-words.txt)
O download do link foi realizado pela biblioteca **requests**. 

In [5]:
def get_analogies(path, file_name, url):

    if not os.path.isfile(path + file_name):
        try:
            r = requests.get(url)
            open(path + file_name , 'wb').write(r.content)
        except (requests.exceptions.ConnectionError, requests.exceptions.HTTPError, 
                requests.exceptions.Timeout, requests.exceptions.RequestException) as err:
            print ("Error Connecting:",err)  
            return None

    data = []
    with open(path + file_name) as fp:
        lines = fp.readlines()
        lines.pop(0)
        for line in lines:
            data.append(line.strip())
    
    return data

analogies = get_analogies('./data/', 'questions-words.txt', 'https://raw.githubusercontent.com/nicholas-leonard/word2vec/master/questions-words.txt')
print(analogies[:10])

['Athens Greece Baghdad Iraq', 'Athens Greece Bangkok Thailand', 'Athens Greece Beijing China', 'Athens Greece Berlin Germany', 'Athens Greece Bern Switzerland', 'Athens Greece Cairo Egypt', 'Athens Greece Canberra Australia', 'Athens Greece Hanoi Vietnam', 'Athens Greece Havana Cuba', 'Athens Greece Helsinki Finland']


## Pré-Processamento de Dados
Nesta etapa, todo o texto é transformado numa lista de palavras e colocado em caixa baixa. Além disso, pontuações, números, caractéres especiais e stopwords foram removidas.

#### Texto convertido para caixa baixa:

In [ ]:
corpus = corpus.lower()
print(corpus[:200])

#### Texto transformado em uma lista de palavras utilizando a biblioteca **NLTK**:

In [ ]:
word_tokens = word_tokenize(corpus)
print(word_tokens[:200])

#### Stopwords, obtidas pelo pacote **NLTK**, foram retiradas do texto:

In [ ]:
stop_words = set(stopwords.words('english'))
filtered_words = [[w for w in word_tokens if not w in stop_words]]
print(filtered_words[0][0:200])

## Treinamento dos Modelos
Em seguida por intermédio da biblioteca gensim os modelos *Continuous Bag of Words* e *Skip-Gram* foram treinados. Como hiperparâmetros dos modelos é interessante destacar os seguintes atributos:
#### Parâmetros fixos

*   epochs = 10
*   min_count = 2
*   report_delay = 1
*   alpha(learning_rate) = 0.01

#### Parâmetros que foram variados
*   vector_size: define a dimensão dos embeddings
*   window: define o tamanho do contexto a ser analizado, ou seja a distância máxima entre a palavra atual e a palavra predita.
*   total_words: Quantidade de palavras utilizadas no treinamento  
*   sg: O algoritmo de treinamento, pode ser tanto o *Continuous Bag of Words*, codificado como 0, ou o *Skip-Gram*, codificado como 1.



In [ ]:
def get_params(vector_size, window, total_words):
    return [list(i) for i in product(vector_size, window, total_words)]

params = get_params(vector_size = [50, 100, 300], window = [3, 6, 10], 
           total_words=[int(len(filtered_words[0])/4), int(len(filtered_words[0])/2), int(len(filtered_words[0]))])
print(params)

#### Função de treinamento dos modelos:
Recebe como parâmetro a dimensão dos embeddings, a janela (ou seja a distância de palavras analisadas), a quantidade de palavras analisadas e o algoritmo (*Skip-Gram* ou *Continuous Bag of Words*)

In [ ]:
def train_model(vector_size, window, total_words, algorithm_name):
    
    algorithm = 1 if "skipgram" else 0
    file_name = "./models/{}/vector_size_{}_window_{}_total_words_{}.sav".format(algorithm_name, vector_size, window, total_words)
    
    if os.path.isfile(file_name):
        print("Modelo com os parâmetros passados já existe em ", file_name)
    else:
        print("Modelo com os seguintes parâmetros será treinado:")
        print("Dimensão do vetor: {} \nWindow: {} \nQuantidade de Palavras: {}\n".format(vector_size, window, total_words))
        
        t = time()
        skip_gram = Word2Vec(filtered_words, vector_size=vector_size, sg=algorithm, window=window, min_count=2, alpha=0.01)
        print('Tempo para construir vocabulario: {} mins'.format(round((time() - t) / 60, 2)))
        
        t = time()
        skip_gram.train(filtered_words, total_examples = total_words, epochs=12, report_delay=1)
        print('Tempo para treinar o modelo: {} mins'.format(round((time() - t) / 60, 2)))

        skip_gram.save(file_name)
        print("Model saved in", file_name)
        
        print("-----------------------------------------------------------------------------------------------------------")

### Treinamento do *Skip-Gram*

In [ ]:
for combinations in params:
    train_model(vector_size=combinations[0], window=combinations[1], total_words=combinations[2], algorithm_name="skipgram")

### Treinamento do *Continuous Bag of Words*

In [ ]:
for combinations in params:
    train_model(vector_size=combinations[0], window=combinations[1], total_words=combinations[2], algorithm_name="cbow")